\tableofcontents

\pagebreak

# Consideraciones Generales

Implementar un canal de mensajería line-delimited JSON entre procesos/dispositivos, con jerarquía de clases para mensajes, serialización/deserialización y un contenedor tabular columnar para el payload. La app se compila con C++17 y organiza headers/definitions con compilación separada.

## Alcance

- Envío y recepción de mensajes por streams estándar (`std::istream/std::ostream`) mediante la clase `Node`.

- Mensajes concretos: `Message` y `AuthMessage` derivados de `IMessage`.

- Serialización: `toJSON()` en cada mensaje; deserialización: `fromJSON(string_view)` tipo factoría.

- DataBag columnar: rows + columns `vector<T>` con `T` $\in$ `{int,double,bool,string}`.

- El framing es “una línea = un mensaje”. Esto replica la pauta de “un objeto por línea”  (en el documento de referencia se enumeran CSV/JSON/XML por línea)


## Hipótesis y supuestos de diseño

- Transporte: lectura con `std::getline`, tolerando `\r\n`. Cada mensaje llega completo por línea.

- Formato: JSON UTF-8, un objeto por línea.

- Errores: si la línea es ilegible, la factoría puede devolver `nullptr` o emitir excepción.
Estas hipótesis reflejan la guía del TP sobre framing por línea y robustez de entrada/salida, ajustadas a nuestro caso exclusivamente JSON

## Principios OO aplicados

- __Encapsulamiento y cohesión__: `Node` solo conoce streams y funciones de serialización.

- __Polimorfismo__: `IMessage` como interfaz; `Message/AuthMessage` sobreescriben `toJSON()`.

- __Compilación separada__: headers livianos, definiciones en `.cpp`. Esto es coherente con los principios señalados en el modelo (encapsulamiento, modularidad, bajo acoplamiento)


# Esquema general de la solución




## Diseño de la solución implementada
Se adoptó una arquitectura simple orientada a objetos, respetando el diagrama de clases provisto en la consigna. El diagrama actualizado se ilustra a continuación:


- `Node`: I/O de líneas. `sendMessage(IMessage&)` escribe `toJSON()` y agrega `'\n'`. `receiveMessage()` hace `getline` y llama a `fromJSON()`.

- `IMessage / AbstractMessage`: exponen `opCode()` y `data()`.

- `Message`: mensaje genérico con `op` y `DataBag`.

- `AuthMessage`: agrega `userID` y `deviceID`.

- `DataBag`: columnar: `rows` y `columns` como `variant<vector<int|double|string|bool>>`.

- `fromJSON()`: factoría mínima que detecta tipo y crea el subtipo correspondiente.

![alt text](Images/TP2Act2.png)


## Diseño de solución secundaria: Intérprete tabular con `Frame` único y `Codec` JSONL

![alt text](Images/Solucion2.png)

### Descripción general

Esta solución elimina la jerarquía de mensajes y el despacho polimórfico. En su lugar, utiliza:

* Un **único tipo de dato** `Frame` que contiene `Meta` y `DataBag`.
* Un **`Codec`** que codifica/decodifica una línea JSON ↔ `Frame`.
* Un **intérprete tabular**: una tabla `handlers[OpCode]` que mapea códigos de operación a funciones manejadoras.
* Una clase **`App`** que orquesta el bucle de lectura, decodificación, ejecución del handler y (opcional) emisión de respuesta.

Se mantiene el protocolo: **JSON por línea** (JSONL), UTF-8, con `data` en formato **columnar**.

### Objetivos

* Reducir al mínimo el número de clases y dependencias.
* Conservar el framing por línea y `DataBag` columnar.
* Lograr ruteo determinista por `OpCode` sin polimorfismo ni `dynamic_cast`.

###  Componentes

* **`App`**: bucle principal, registro de handlers, E/S por streams.
* **`Codec`**: `encode(const Frame&) -> std::string` y `decode(std::string_view) -> Frame`.
* **`Frame`**: estructura con `Meta meta; DataBag data;`.
* **`Meta`**: `type`, `op` (compatible con `enum class OpCode:int`), versión, `msgId`, `ts`.
* **`DataBag`**: bloque columnar (`rows` + `columns<string → vector<T>>`, con `T ∈ {int,double,string,bool}`).
* **Tabla de handlers**: `std::unordered_map<OpCode, Handler>` donde `Handler` es `std::function<void(const Frame&, App&)>`.

###  Contrato de mensaje (resumen)

* **Framing**: 1 línea = 1 mensaje (`\n`; se tolera `\r\n`).
* **Campos mínimos**:

  * `meta.type`: `"auth"`, `"message"`, etc.
  * `meta.op`: entero mapeable a `OpCode`.
* **`data` opcional (columnar)**:

  ```json
  "data": {
    "rows": 3,
    "columns": {
      "id":   [1,2,3],
      "ok":   [true,false,true]
    }
  }
  ```

### Flujo de operación

1. `App` lee una línea del `istream`.
2. `Codec.decode` produce un `Frame`.
3. `App` consulta `handlers[frame.meta.op]` y ejecuta la función registrada.
4. El handler, si corresponde, construye un `Frame` de respuesta y llama a `App.send(frameResp)`, que usa `Codec.encode` y escribe una línea al `ostream`.

### Ventajas y limitaciones

* **Ventajas**: muy pocas clases, cero herencia, cero conversión en tiempo de ejecución; testeo y extensión por tabla (`registerHandler`).
* **Limitaciones**: no hay validaciones transversales integradas (versionado/autorización); si se requieren, se agregan en `App` antes del despacho.

### Párrafo breve para el informe

La variante C-min reemplaza la jerarquía polimórfica por un **intérprete tabular**: un `Frame` único encapsula `Meta` y `DataBag`, y una tabla de handlers asocia cada `OpCode` con su lógica. `App` realiza el bucle de lectura y despacho, mientras que `Codec` se encarga de la (de)serialización JSON por línea. El resultado preserva el contrato de intercambio y simplifica la estructura a tres clases principales más una tabla de funciones, manteniendo el diseño orientado a objetos donde aporta valor (orquestación y encapsulamiento), pero evitando herencia y conversiones dinámicas.


# Interfaces de usuario


## Descripción general de la Interfaz de Usuario

La aplicación no dispone de interfaz gráfica. La interacción se realiza mediante:

* **Consola (STDIN/STDOUT/STDERR)** para procesamiento por lotes o encadenamiento con otros procesos.
* **Archivos** en formato JSON Lines (`.jsonl`) para entradas y salidas persistentes.
* **Puerto serie** para intercambio con dispositivos embebidos (por ejemplo, microcontroladores).

El sistema opera en modo **no interactivo**: procesa una secuencia de mensajes, uno por línea, y termina al alcanzar EOF o un error fatal.

## Modalidades de uso

### Modo tubería (pipeline)

Permite integrar la aplicación en cadenas de procesamiento.

```bash
productor | ./tp2_actividad2 | consumidor
```

* Entrada y salida son flujos de texto UTF-8.
* Cada línea corresponde a un mensaje JSON completo.

### Modo archivo

Adecuado para pruebas reproducibles o integración con datasets.

```bash
./tp2_actividad2 < input.jsonl > output.jsonl
```

* `input.jsonl`: un objeto JSON por línea.
* `output.jsonl`: una línea por mensaje producido.

### Modo puerto serie (Linux)

Intercambio con el dispositivo a través de `stdin/stdout` redirigidos al TTY.

```bash
stty -F /dev/ttyACM0 115200 cs8 -parenb -cstopb -echo -icanon -ixon -ixoff -crnl
./tp2_actividad2 < /dev/ttyACM0
# o envío de una línea JSON
printf '%s\n' '{"type":"auth","op":2,"userID":"u","deviceID":"esp32"}' > /dev/ttyACM0
```

* Requisito: el firmware debe emitir **un `\n` por mensaje**. Se tolera `\r\n` y se normaliza internamente.

## Convenciones de entrada/salida

* **Framing:** 1 línea = 1 mensaje. El emisor no debe incluir `\n` dentro del objeto; la aplicación agrega `\n` al enviar y usa `getline` al recibir.
* **Codificación:** UTF-8.
* **Formato de mensaje:** objeto JSON con campos mínimos `type` y `op`, y opcional `data` (bloque tabular columnar).
* **Buffering:** la aplicación no fuerza `flush` continuo; en pipelines, el flushing lo gestiona el SO. Para diagnóstico en tiempo real, el consumidor puede forzar `std::flush` o usar `stdbuf -oL`.


# Recursos adicionales

No se hace uso de ningun componente de software no estandar del lenguaje ni de la plataforma para codificar el programa. A continuación, se listan las librerias estandar de C++ y de Linux que se utilizan en los diferentes archivos `.h` y `.cpp`.

```C++
#include <istream>
#include <ostream>
#include <sstream>
#include <memory>
#include <stdexcept>
#include <cstddef>
#include <utility>

// Tipos de variables
#include <string>
#include <vector>
#include <variant>
#include <unordered_map>

```

# Manual de instrucciones de la aplicación

## Requisitos del entorno

Compilador C++ con soporte para C++17 (por ejemplo, g++ ≥ 9).

Herramienta de construcción make si se utiliza el Makefile provisto.

Sistema operativo tipo POSIX para pruebas con puerto serie (ej.: GNU/Linux).

## Compilación

### Con Makefile

```bash
make            # compila el proyecto
make clean      # elimina artefactos de compilación
```
![alt text](Images/CrearAplicacion.png)


Parámetros esperados del compilador: `-std=c++17 -Wall -Wextra -Wpedantic`

### Línea de comandos (alternativa)
```bash
g++ -std=c++17 -Wall -Wextra -Iinclude \
  src/Node.cpp src/Message.cpp src/AuthMessage.cpp src/DataBag.cpp src/Serializer.cpp \
  src/main.cpp -o tp2_actividad2
```
![alt text](Images/CreacAplicacion2.png)

## Ejecución Básica (STDIN/STDOUT)

La aplicación procesa un mensaje por línea en formato JSON (JSON Lines).

- Entrada: `stdin` (una línea JSON por mensaje, terminada en `\n`).
- Salida: `stdout` (una línea JSON por mensaje producido).

Por ejemplo:

```bash
printf '%s\n' \
'{"type":"message","op":42}' \
'{"type":"auth","op":7,"userID":"u","deviceID":"d"}' \
| ./tp2_actividad2
```

La aplicación leerá cada línea, la deserializará y la procesará de acuerdo con su tipo.

## Contrato del mensaje (formato)

- Framing: 1 línea = 1 mensaje. Fin de línea `\n`. Se acepta `\r\n` y se normaliza a `\n`.

- Codificación: UTF-8.

- Campos obligatorios:

    - `type`: `"message"` o `"auth"`.

    - `op`: entero que identifica la operación (código de operación).

- Campo opcional:

    - `data`: bloque tabular columnar.

### Estructura del bloque `data` (columnar)

```JSON
"data": {
  "rows": 3,
  "columns": {
    "id":   [1, 2, 3],
    "temp": [21.5, 22.0, 23.1],
    "ok":   [true, false, true],
    "name": ["a", "b", "c"]
  }
}
```

Invariante: para toda columna `columns[k], columns[k].size() == rows`.


## Flujo de operación

### Envío

1. El emisor construye el objeto de mensaje.
2. `toJSON()` genera una cadena **sin** salto de línea.
3. `Node::sendMessage(const IMessage&)` escribe la cadena y agrega `'\n'` al flujo de salida.

### Recepción

1. `Node::receiveMessage()` lee una línea mediante `std::getline`.
2. Se elimina un `'\r'` final si existiese (`\r\n`).
3. `fromJSON(std::string_view)` decide el subtipo (`Message` o `AuthMessage`) y construye el objeto resultante.
4. En caso de EOF o error de lectura, `receiveMessage()` retorna `nullptr`.

## Integración con archivos y tuberías

* Lectura desde archivo:

  ```bash
  ./tp2_actividad2 < input.jsonl > output.jsonl
  ```
* Composición con otros procesos:

  ```bash
  productor | ./tp2_actividad2 | consumidor
  ```

## Uso con puerto serie (GNU/Linux)

### Detección y configuración

Identificar el dispositivo (p. ej., `/dev/ttyACM0` o `/dev/ttyUSB0`) y configurar el modo raw:

```bash
stty -F /dev/ttyACM0 115200 cs8 -parenb -cstopb \
     -echo -icanon -ixon -ixoff -crnl
```

* Velocidad: 115200 baudios (ajustable).
* Formato: 8N1 (8 bits de datos, sin paridad, 1 bit de stop).
* Desactivación de eco, modo canónico y control de flujo por software.

### Intercambio de mensajes

* Enviar al dispositivo:

  ```bash
  printf '%s\n' '{"type":"auth","op":2,"userID":"u","deviceID":"esp32"}' > /dev/ttyACM0
  ```
* Recibir desde el dispositivo:

  ```bash
  ./tp2_actividad2 < /dev/ttyACM0
  ```

**Requisito**: el firmware debe transmitir **una línea JSON por mensaje**. Se admiten finales `\r\n`.



## Pruebas y validación

### Prueba de humo (JSON Lines)

Entrada:

```
{"type":"message","op":42}
{"type":"auth","op":7,"userID":"u","deviceID":"d"}
```

Procedimiento:

```bash
printf '%s\n' \
'{"type":"message","op":42}' \
'{"type":"auth","op":7,"userID":"u","deviceID":"d"}' \
| ./tp2_actividad2
```

Criterio de aceptación: cada línea se procesa sin error; el programa mantiene el framing y reconoce el subtipo.

![alt text](Images/PruebaDeHumo.png)

### Verificación de `DataBag`

1. Construir un `DataBag` con 2–4 columnas y `rows > 0`.
2. Enviar un `Message` con `data` y comprobar que la serialización JSON cumple el invariante de tamaños.
3. Probar `selectRows` y `selectColumns` con índices válidos y detectar errores ante índices fuera de rango.

### Normalización de finales de línea

Probar entradas con `\n` y `\r\n`. Criterio: el receptor debe aceptar ambos y operar internamente con `\n`.



# Conclusiones

La solución implementada cumple con los objetivos establecidos: define un protocolo de intercambio basado en JSON line-delimited (un mensaje por línea) y una jerarquía de clases que separa responsabilidades entre interfaz (`IMessage`), factor común (`AbstractMessage`) y tipos concretos (`Message`, `AuthMessage`). La clase `Node` encapsula la E/S sobre `std::istream/std::ostream`, garantizando el enmarcado por línea en el envío (`sendMessage` agrega el salto de línea) y la lectura robusta en la recepción (`receiveMessage` normaliza `\r\n` y delega la construcción a la factoría `fromJSON`).

El campo `op` opera como código de operación y permite el ruteo determinista del flujo de control en el receptor, mientras que el bloque `data` encapsula el payload en un modelo tabular columnar. Esta representación columnar, sustentada en vectores tipados (`int`, `double`, `bool`, `string`), asegura coherencia estructural mediante el invariante `|columna| = rows` y optimiza el acceso secuencial a datos. Las operaciones de proyección y filtrado (`selectColumns`, `selectRows`) se implementan sin romper dicho invariante.

Las pruebas de humo demuestran el funcionamiento extremo a extremo: envío y recepción por líneas, identificación polimórfica del tipo de mensaje y serialización consistente. La arquitectura resultante es extensible: incorporar nuevos tipos de mensajes o columnas requiere cambios localizados (un subtipo adicional y su caso en la factoría), sin afectar el resto del sistema. Como trabajo futuro, se recomienda: 
    
1) completar el parseo inverso del bloque `data` hacia `DataBag` con validación de tipos y tamaños, 
    
2) formalizar un esquema de columnas por operación (`op`) para verificación temprana, y 
    
3) parametrizar políticas de tiempo de espera, reintentos y manejo de errores en `Node` según el medio de transporte (archivo, tubería o puerto serie).


# Referencias consultadas

- Apuntes de cátedra de POO (sección “Guía de Trabajos Prácticos”).

- 115 Ejercicios resueltos de programación C++(Ra-Ma) by Joefebeus & Iryopogu


# Anexo
